# CONFIGURAÇÃO DO AMBIENTE

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=3f7b473108d066a40c90b7601b940fff9c184402ae07cc4c12dee3756a0e233d
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
!pip list

Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         0.34.2
aiohappyeyeballs                   2.4.3
aiohttp                            3.10.10
aiosignal                          1.3.1
alabaster                          0.7.16
albucore                           0.0.16
albumentations                     1.4.15
altair                             4.2.2
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.5.1
arviz                              0.19.0
astropy                            6.1.4
astropy-iers-data                  0.2024.10.7.0.32.46
astunparse                         1.6.3
async-timeout                      4.0.3
atpublic                           4.1.0
attrs                              24.2.0
audioread        

In [ ]:
import pyspark
print(pyspark.__version__)

3.5.3


In [ ]:
# importando o spark, sempre precisamos do "SparkSession"
from pyspark.sql import SparkSession

In [ ]:
# ativando o auto-completar
%config Completer.use_jedi = True

In [ ]:
from pyspark.sql import SparkSession

# Criando uma sessão Spark
spark = SparkSession.builder.appName("MeuApp").getOrCreate()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

# Importação Marchine Learning

In [ ]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor

# avaliar a performace do modelo para regressão
from pyspark.ml.evaluation import RegressionEvaluator

# tratamento de dados - vai transformar os dados categoricos em dados numericos
# tb cria a coluna com tds os atributos
from pyspark.ml.feature import VectorAssembler

# Carregando e preparando dados

In [ ]:
Carros_temp = spark.read.csv("/content/Carros.csv", header=True, inferSchema=True, sep=";")
Carros_temp.show(5)

In [ ]:
# novo dataframe com as colunas q seram usadas
# variaveis independentes para prever o 'HP'
Carros = Carros_temp.select("Consumo", "Cilindros", "Cilindradas", "HP")
Carros.show(5)

In [ ]:
# Criando 'vetor' de caracteristicas = coluna com as variaveis independentes
# inputColumns - variaveis q sao independentes | outputCol - nome q a coluna sera gerada

veccaracteristicas = VectorAssembler(inputCols=[("Consumo"), ("Cilindros"), ("Cilindradas")], outputCol="caracteristicas")

In [ ]:
# aplicando o obj 'vector' ao 'carros'
Carros = veccaracteristicas.transform(Carros)
Carros.show(5)

In [ ]:
# dividir os dados em treino e testes
# randomSplit ([70% para treino, 30% para testes])
CarrosTreino, CarrosTeste = Carros.randomSplit([0.7, 0.3])

# verificando
print(CarrosTreino.count())
print(CarrosTeste.count())

# Criando o modelo de Regressão Linear

In [ ]:
# criando obj do tipo Regressão Linear
# paramentros: coluna independente, coluna dependente
reglin = LinearRegression(featuresCol="caracteristicas", labelCol="HP")

In [ ]:
# treino
# gerando o modelo
modelo = reglin.fit(CarrosTreino)

In [ ]:
# modelo criado
# modelo - aquilo q o algoritmo cria baseado nos dados para fazer previsões
# ou seja, não precisamos mais do "linearRegression" e dos 'dados'
# precisamos agora apenas do modelo

In [ ]:
# fazendo previsão com os dados de testes
previsao = modelo.transform(CarrosTeste)
previsao.show(5)
# metodos de 'prevsiao' retornam um 'dataframe'

In [ ]:
# avaliando o modelo
# metrica de performace
# paramentros: coluna de predicti, coluna de como realmente aconteceu (classe), metrica q vamos utilizar
avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="HP", metricName="rmse")


In [ ]:
# avaliando a performace do obj q foi previsto
rmse = avaliar.evaluate(previsao)
print(rmse)

In [ ]:
# gerando outro modelo
# fazer previsao com dados de teste
# gera a metrica rmse
# vamos comparar com a metrica ja vista

In [ ]:
rfreg = RandomForestRegressor(featuresCol="caracteristicas", labelCol="HP")

In [ ]:
modelo2 = rfreg.fit(CarrosTreino)

In [ ]:
previsao2 = modelo2.transform(CarrosTeste)
previsao2.show(5)

In [ ]:
# comparando
previsao.show(5)
previsao2.show(5)

In [ ]:
# gerando metrica para segunda previsao
rmse2 = avaliar.evaluate(previsao2)
print(rmse2)

In [ ]:
# comparando
print(rmse)
print(rmse2)
# quanto menor, melhor!